In [ ]:
!git clone --recursive https://github.com/Microsoft/LightGBM
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

Cloning into 'LightGBM'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 20629 (delta 47), reused 37 (delta 15), pack-reused 20531
Receiving objects: 100% (20629/20629), 16.42 MiB | 14.15 MiB/s, done.
Resolving deltas: 100% (15063/15063), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728        
Receiving objects: 100% (21728/21728)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import DataFrame, concat
import math
from math import radians
from google.colab import drive
drive.mount('/content/drive')
from datetime import date
import lightgbm as lgbm
from lightgbm import LGBMRegressor
import time
import statsmodels.api as sm
from sklearn import linear_model, metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.regression.quantile_regression import QuantReg
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# 데이터 불러오기 

raw_data = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/train/train.csv')
submission = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/sample_submission.csv')
submission.shape

(7776, 10)

In [ ]:
#### 파라미터 ####

# 하루의 틱
ticks = 48

# 예측에 사용할 일수
days = 3                # 이 부분을 바꿀 수 있다.
n_days = ticks*days

# 미래 예측할 일수
future_days = 2
future_window = ticks * future_days

### 모든변수
# ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']

# 사용할 변수
what_to_left = ['Time', 'WS', 'DHI','DNI','RH','T','TARGET', 'Day']

n_features = len(what_to_left)
n_obs = n_days * n_features # 7일 예측 기준으로 7(일 수) * 48(틱 수) * 6(변수 개수)

In [ ]:
def drop_clms(dataset):
  dataset['Time'] = dataset['Hour'] + dataset['Minute']*(0.5/30)
  dataset['Date'] = dataset['Day']%365
  dataset.drop('Day', axis=1, inplace=True)
  
  temp = list()
  for i in range(0, len(dataset), 48):
    temp += [24-(list(dataset.DHI[i:i+48]).count(0)*0.5)]*48 

  dataset["SH"] = temp

  return dataset

def cos_time(dataset):
  dataset['sin_time'] = np.sin(2*np.pi*dataset.Time/24)
  dataset['cos_time'] = np.cos(2*np.pi*dataset.Time/24)
  return dataset

def get_yday(when):
  HP = date(2020,1,1)
  results = (when-HP).days
  return results

def 절기24(Date):
  target = 0
  for i in list24:
    if Date < i:
      target = list24.index(i) - 1
      break
  if Date < 5:
    target = 23
  return target

In [ ]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def train_to_supervised(train, n_in):

    clmns = list(train.columns)

    # 타켓 칼럼의 이름을 여기에 입력
    target = ['TARGET']

    # 클래스 변수로 전환(encoding)할 칼럼을 여기에 입력
    class_lst = ['season']

    # 기타 칼럼은 전과 같이 들어갈 것.
    scaled_lst = list(set(clmns) - set(class_lst))

    scaled_df = train[scaled_lst]
    class_df = train[class_lst]
    target_df = train[target]

    # 미래 몇 번째 항목을 가져올 것인가
    future = [48, 96]

    ### 만약에 스케일링을 하고 싶다면 ###
    # scaled_df 데이터 프레임만 스케일링 하고, 절기랑 TARGET 데이터는 그냥 두면 된다.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    n_vars2 = 1 if type(target_df) is list else target_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]

    # class라 encoding 필요한 절기 끝에 하나만 붙여놓기
    cols.append(class_df)
    names += class_lst

    # 48과 96 후의 타겟 데이터 2개 붙이기.
    # forecast sequence (t, t+1, ... t+n)
    for i in future:
        cols.append(target_df.shift(-i))
        if i == 0:
            names += [('TARGET%d(t)' % (j+1)) for j in range(n_vars2)]
        else:
            names += [('TARGET%d(t+%d)' % (j+1, i)) for j in range(n_vars2)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [ ]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def test_to_supervised(train, n_in):

    clmns = list(train.columns)

    # 타켓 칼럼의 이름을 여기에 입력
    target = ['TARGET']

    # 클래스 변수로 전환(encoding)할 칼럼을 여기에 입력
    class_lst = ['season']

    # 기타 칼럼은 전과 같이 들어갈 것.
    scaled_lst = list(set(clmns) - set(class_lst))

    scaled_df = train[scaled_lst]
    class_df = train[class_lst]

    ### 만약에 스케일링을 하고 싶다면 ###
    # testset의 스케일링은 구조가 상당히 까다로우므로....... 일단 나중에 하기로 함.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]

    # class라 encoding 필요한 절기 끝에 하나만 붙여놓기
    cols.append(class_df)
    names += class_lst
    
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [ ]:
소한 = get_yday(date(2020,1,6))
대한 = get_yday(date(2020,1,20))
입춘 = get_yday(date(2020,2,4))
우수 = get_yday(date(2020,2,19))
경칩 = get_yday(date(2020,3,6))
춘분 = get_yday(date(2020,3,21))
청명 = get_yday(date(2020,4,5))
곡우 = get_yday(date(2020,4,20))
입하 = get_yday(date(2020,5,6))
소만 = get_yday(date(2020,5,21))
망종 = get_yday(date(2020,6,6))
하지 = get_yday(date(2020,6,22))
소서 = get_yday(date(2020,7,7))
대서 = get_yday(date(2020,7,23))
입추 = get_yday(date(2020,8,8))
처서 = get_yday(date(2020,8,23))
백로 = get_yday(date(2020,9,8))
추분 = get_yday(date(2020,9,23))
한로 = get_yday(date(2020,10,8))
상강 = get_yday(date(2020,10,24))
입동 = get_yday(date(2020,11,8))
소설 = get_yday(date(2020,11,22))
대설 = get_yday(date(2020,12,7))
동지 = get_yday(date(2020,12,22))

list24 = [소한, 대한, 입춘, 우수, 경칩, 춘분, 청명, 곡우, 입하, 소만, 망종, 하지, 소서, 대서, 입추, 처서, 백로, 추분, 한로, 상강, 입동, 소설, 대설, 동지]

In [ ]:
m = 60

일출 = [7+47/m, 7+44/m, 7+34/m, 7+18/m, 6+56/m, 6+35/m, 6+12/m, 5+51/m, 5+33/m, 5+19/m, 5+11/m, 5+11/m, 5+17/m, 5+27/m, 5+41/m, 5+54/m, 6+8/m, 6+20/m, 6+33/m, 6+47/m, 7+3/m, 7+20/m, 7+33/m, 7+45/m]
일몰 = [17+28/m, 17+42/m, 17+58/m, 18+15/m, 18+30/m, 18+44/m, 18+58/m, 19+12/m, 19+25/m, 19+39/m, 19+50/m, 19+57/m, 19+56/m, 19+50/m, 19+35/m, 19+15/m, 18+51/m, 18+28/m, 18+6/m, 17+45/m, 17+28/m, 17+17/m, 17+13/m, 17+17/m]
경사각 = [32.92, 36.83, 40.75, 44.67, 48.58, 52.5, 56.42, 60.33, 64.25, 68.16, 72.01, 76, 72.1, 68.16, 64.25, 60.33, 56.42, 52.5, 48.58, 44.67, 40.75, 36.83, 32.92, 29]
남중 = [12+36/m, 12+41/m, 12+44/m, 12+44/m, 12+42/m, 12+38/m, 12+34/m, 12+29/m, 12+27/m, 12+27/m, 12+29/m, 12+32/m, 12+35/m, 12+37/m, 12+36/m, 12+33/m, 12+29/m, 12+23/m, 12+18/m, 12+15/m, 12+14/m, 12+16/m, 12+22/m, 12+29/m]
print(len(일출), len(일몰), len(경사각), len(남중))

24 24 24 24


In [ ]:
def HRA(DHI, DNI, season, hour):
  # 위도(latitude) 기준을 일단 임의로 대전으로 설정 (위도 36.19~36.2도)
  latitude = radians(36.2)
  season = int(season)
  # 절기별 대한민국의 경사각
  tilt = radians(경사각[season])

  # 절기별 대한민국 대전의 태양 남중시각
  hra = radians(15*(hour - 남중[season]))

  # 구하려는 알파
  elevation = np.arcsin(np.sin(tilt) * np.sin(latitude) + np.cos(tilt) * np.cos(latitude) * np.cos(hra))

  # 천정각(Zenith Angle)은 90 - 알파
  zenith = radians(90) - elevation

  # GHI는 DHI + DNI * cos(천정각)
  ghi = DHI + DNI *np.cos(zenith)

  return ghi

In [ ]:
def not_minus(dataset):
  for i in range(0, len(dataset.index)):
    for j in range(0, len(dataset.columns)):
      K = dataset.iloc[i,j]
      if K < 0:
        dataset.iloc[i,j] = 0
  return dataset

In [ ]:
# 차원축소
# 원하는 변수의 숫자만큼 남긴다.
# dimension 변수에 원하는 변수의 숫자를 적어준다.

def alzip(dataset, dimension):
    X = dataset.iloc[:,:-1]
    y = dataset.iloc[:,-1:]
    pca = PCA(n_components= dimension)
    pca.fit(X)
    pica_X = pd.DataFrame(pca.transform(X), index = X.index)
    dataset2 = pd.concat([pica_X, y], axis = 1)
    return dataset2

In [ ]:
dataset = raw_data.copy()
dataset2 = drop_clms(dataset)

dataset2.drop(['Hour', 'Minute'], axis =1, inplace=True)
dataset2['season'] = dataset2.apply(lambda x: 절기24(x['Date']), axis = 1)

dataset3 = cos_time(dataset2)
dataset4 = dataset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]

dataset5= dataset4.copy()
dataset5['GHI'] = dataset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)

dataset6 = dataset5[['DHI', 'DNI', 'WS', 'RH', 'T','SH', 'sin_time', 'cos_time', 'GHI', 'season', 'TARGET']]
dataset7 = train_to_supervised(dataset6, n_days)

X = dataset7.iloc[:,:-2]
y_1 = pd.DataFrame(dataset7.iloc[:,-2])
y_2 = pd.DataFrame(dataset7.iloc[:,-1])
print(len(X))
print(len(X.columns))
X.drop('season', axis=1, inplace=True)

52320
1441


In [ ]:
df_test = []
df_valid_X = []
df_valid_y1 = []
df_valid_y2 = []
# 예시 코드
# 아직 유진이 어떻게 변수 추가할 지 몰라서 to_supervised는 쓰지 않았음.


for i in range(81):
    file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(i) + '.csv'
    new_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/adj_test/' + str(i) + '.csv'
    
    # adj_test에서 가져와야 하므로 new_path에서 불러온다
    temp = pd.read_csv(new_path)
    temp.drop('Unnamed: 0', axis = 1, inplace = True)
    testset = temp.copy()

    # 24로 저장한 24절기 season으로 이름 바꾸기
    testset.rename(columns = {'24' : 'season'}, inplace = True)

    # 이 다음은 trainset과 동일한 전처리, season은 이미 구해져 있으므로 구하지 않음
    testset2 = drop_clms(testset)
    testset2.drop(['Hour', 'Minute','Date'], axis=1,inplace=True)
    testset3 = cos_time(testset2)
    testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]

    testset5 = testset4.copy()
    testset5['GHI'] = testset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)
    testset6 = testset5[['DHI', 'DNI', 'WS', 'RH', 'T','SH', 'sin_time', 'cos_time', 'GHI', 'season', 'TARGET']]

    #############################
    validset = train_to_supervised(testset6, n_days)
    X_valid = validset.iloc[:,:-2]
    X_valid.drop('season', axis =1, inplace=True)

    valid_y_1 = pd.DataFrame(validset.iloc[:,-2])

    df_valid_X.append(X_valid)
    df_valid_y1.append(valid_y_1)
    ############################

X_valid_1t = pd.concat(df_valid_X)
Y_valid_1t = pd.concat(df_valid_y1)
X_valid_1t

,WS(t-144),GHI(t-144),DHI(t-144),T(t-144),cos_time(t-144),sin_time(t-144),DNI(t-144),TARGET(t-144),RH(t-144),SH(t-144),WS(t-143),GHI(t-143),DHI(t-143),T(t-143),cos_time(t-143),sin_time(t-143),DNI(t-143),TARGET(t-143),RH(t-143),SH(t-143),WS(t-142),GHI(t-142),DHI(t-142),T(t-142),cos_time(t-142),sin_time(t-142),DNI(t-142),TARGET(t-142),RH(t-142),SH(t-142),WS(t-141),GHI(t-141),DHI(t-141),T(t-141),cos_time(t-141),sin_time(t-141),DNI(t-141),TARGET(t-141),RH(t-141),SH(t-141),...,WS(t-4),GHI(t-4),DHI(t-4),T(t-4),cos_time(t-4),sin_time(t-4),DNI(t-4),TARGET(t-4),RH(t-4),SH(t-4),WS(t-3),GHI(t-3),DHI(t-3),T(t-3),cos_time(t-3),sin_time(t-3),DNI(t-3),TARGET(t-3),RH(t-3),SH(t-3),WS(t-2),GHI(t-2),DHI(t-2),T(t-2),cos_time(t-2),sin_time(t-2),DNI(t-2),TARGET(t-2),RH(t-2),SH(t-2),WS(t-1),GHI(t-1),DHI(t-1),T(t-1),cos_time(t-1),sin_time(t-1),DNI(t-1),TARGET(t-1),RH(t-1),SH(t-1)
144,2.7,0.0,0.0,0.0,1.000000,0.000000,0.0,0.0,34.42,9.5,2.7,0.0,0.0,0.1,0.991445,0.130526,0.0,0.0,34.17,9.5,2.7,0.0,0.0,0.2,0.965926,0.258819,0.0,0.0,34.23,9.5,2.7,0.0,0.0,0.3,0.923880,0.382683,0.0,0.0,33.99,9.5,...,1.2,0.0,0.0,0.4,0.866025,-0.500000,0.0,0.0,54.54,9.5,1.2,0.0,0.0,0.5,0.923880,-0.382683,0.0,0.0,54.08,9.5,1.2,0.0,0.0,0.5,0.965926,-0.258819,0.0,0.0,55.85,9.5,1.2,0.0,0.0,0.3,0.991445,-0.130526,0.0,0.0,56.59,9.5
145,2.7,0.0,0.0,0.1,0.991445,0.130526,0.0,0.0,34.17,9.5,2.7,0.0,0.0,0.2,0.965926,0.258819,0.0,0.0,34.23,9.5,2.7,0.0,0.0,0.3,0.923880,0.382683,0.0,0.0,33.99,9.5,2.8,0.0,0.0,0.4,0.866025,0.500000,0.0,0.0,33.97,9.5,...,1.2,0.0,0.0,0.5,0.923880,-0.382683,0.0,0.0,54.08,9.5,1.2,0.0,0.0,0.5,0.965926,-0.258819,0.0,0.0,55.85,9.5,1.2,0.0,0.0,0.3,0.991445,-0.130526,0.0,0.0,56.59,9.5,1.1,0.0,0.0,0.1,1.000000,0.000000,0.0,0.0,59.09,9.5
146,2.7,0.0,0.0,0.2,0.965926,0.258819,0.0,0.0,34.23,9.5,2.7,0.0,0.0,0.3,0.923880,0.382683,0.0,0.0,33.99,9.5,2.8,0.0,0.0,0.4,0.866025,0.500000,0.0,0.0,33.97,9.5,2.9,0.0,0.0,0.6,0.793353,0.608761,0.0,0.0,33.48,9.5,...,1.2,0.0,0.0,0.5,0.965926,-0.258819,0.0,0.0,55.85,9.5,1.2,0.0,0.0,0.3,0.991445,-0.130526,0.0,0.0,56.59,9.5,1.1,0.0,0.0,0.1,1.000000,0.000000,0.0,0.0,59.09,9.5,1.2,0.0,0.0,-0.4,0.991445,0.130526,0.0,0.0,61.20,9.5
147,2.7,0.0,0.0,0.3,0.923880,0.382683,0.0,0.0,33.99,9.5,2.8,0.0,0.0,0.4,0.866025,0.500000,0.0,0.0,33.97,9.5,2.9,0.0,0.0,0.6,0.793353,0.608761,0.0,0.0,33.48,9.5,3.0,0.0,0.0,0.7,0.707107,0.707107,0.0,0.0,33.97,9.5,...,1.2,0.0,0.0,0.3,0.991445,-0.130526,0.0,0.0,56.59,9.5,1.1,0.0,0.0,0.1,1.000000,0.000000,0.0,0.0,59.09,9.5,1.2,0.0,0.0,-0.4,0.991445,0.130526,0.0,0.0,61.20,9.5,1.3,0.0,0.0,-0.8,0.965926,0.258819,0.0,0.0,65.76,9.5
148,2.8,0.0,0.0,0.4,0.866025,0.500000,0.0,0.0,33.97,9.5,2.9,0.0,0.0,0.6,0.793353,0.608761,0.0,0.0,33.48,9.5,3.0,0.0,0.0,0.7,0.707107,0.707107,0.0,0.0,33.97,9.5,3.1,0.0,0.0,0.9,0.608761,0.793353,0.0,0.0,33.44,9.5,...,1.1,0.0,0.0,0.1,1.000000,0.000000,0.0,0.0,59.09,9.5,1.2,0.0,0.0,-0.4,0.991445,0.130526,0.0,0.0,61.20,9.5,1.3,0.0,0.0,-0.8,0.965926,0.258819,0.0,0.0,65.76,9.5,1.4,0.0,0.0,-0.9,0.923880,0.382683,0.0,0.0,66.24,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,1.2,0.0,0.0,9.3,0.793353,-0.608761,0.0,0.0,68.24,15.0,1.2,0.0,0.0,8.7,0.866025,-0.500000,0.0,0.0,69.78,15.0,1.3,0.0,0.0,8.3,0.923880,-0.382683,0.0,0.0,71.70,15.0,1.4,0.0,0.0,7.9,0.965926,-0.258819,0.0,0.0,71.52,15.0,...,0.8,0.0,0.0,15.9,0.382683,-0.923880,0.0,0.0,54.63,15.0,0.9,0.0,0.0,15.2,0.500000,-0.866025,0.0,0.0,54.55,15.0,0.9,0.0,0.0,14.5,0.608761,-0.793353,0.0,0.0,57.06,15.0,1.0,0.0,0.0,13.8,0.707107,-0.707107,0.0,0.0,58.70,15.0
236,1.2,0.0,0.0,8.7,0.866025,-0.500000,0.0,0.0,69.78,15.0,1.3,0.0,0.0,8.3,0.923880,-0.382683,0.0,0.0,71.70,15.0,1.4,0.0,0.0,7.9,0.965926,-0.258819,0.0,0.0,71.52,15.0,1.5,0.0,0.0,7.6,0.991445,-0.130526,0.0,0.0,72.99,15.0,...

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
min = 99999999999999999999999 
count = 0 #Used for keeping track of the iteration number
#How many runs to perform using randomly selected hyperparameters
iterations = 200

X_test = X_valid_1t.copy()
y_test = Y_valid_1t.copy()
X_set = X.copy()

pp = {}
for i in range(iterations):
    print('iteration number', count)
    count += 1 #increment count

    params = {} #initialize parameters
    params['learning_rate'] = np.random.uniform(0, 1)
    params['boosting_type'] = np.random.choice(['gbdt', 'dart', 'goss'])
    params['objective'] = 'quantile'
    params['alpha'] = 0.5
    params['device'] = 'gpu'
    params['sub_feature'] = np.random.uniform(0, 1)
    params['num_leaves'] = np.random.randint(20, 300)
    params['min_data'] = np.random.randint(10, 100)
    params['max_depth'] = np.random.randint(5, 200)
    params['random_state'] = 42
    params['n_estimators'] = 500
    params['bagging_fraction'] = np.random.uniform(0,1)
    params['feature_fraction'] = np.random.uniform(0,1)

    Pika = np.random.randint(80,100)*0.01
    pca = PCA(n_components= Pika)
    pca.fit(X_set)
    pika_X = pd.DataFrame(pca.transform(X_set), index = X_set.index)
    test_X = pd.DataFrame(pca.transform(X_test), index = X_test.index)

    X_train_1, X_valid_1, Y_train_1,  Y_valid_1 = train_test_split(pika_X, y_1, test_size=0.30, random_state=42)

    model = LGBMRegressor(**params)
    model.fit(X_train_1, Y_train_1, eval_metric = ['quantile'], eval_set=[(X_valid_1, Y_valid_1)], verbose=500)

    y_pred=model.predict(test_X)
    mae=mean_absolute_error(y_pred,y_test)
    print('MAE:', mae)
    if mae < min:
      min = mae
      pp = params 
      best_model = model

print(pp)
print(best_model)
print(Pika)

iteration number 0
[LightGBM] [Warning] min_data_in_leaf is set with min_data=70, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=70
[LightGBM] [Warning] feature_fraction is set with sub_feature=0.9340706900499288, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9340706900499288
[LightGBM] [Warning] feature_fraction is set=0.7853050053217401, sub_feature=0.9340706900499288 will be ignored. Current value: feature_fraction=0.7853050053217401
[LightGBM] [Warning] bagging_fraction is set=0.44178530952636397, subsample=1.0 will be ignored. Current value: bagging_fraction=0.44178530952636397
[500]	valid_0's quantile: 2.78132
MAE: 7.065579348331695
iteration number 1
[LightGBM] [Warning] min_data_in_leaf is set with min_data=44, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=44
[LightGBM] [Warning] feature_fraction is set with sub_feature=0.7262997311293733, colsample_bytree=1.0 will be ignored. Current value: feature_fracti